# 전체 모델 빌딩 파이프라인 개발 (SageMaker Model Building Pipeline 모든 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 **약 20분** 소요 됩니다.

- 1. SageMaker 모델 빌드 파이프라인을 이용한 모델 빌드 오케스트레이션
- 2. 파이프라인 개발자 가이드
- 3. 기본 라이브러리 로딩
- 4. 모델 빌딩 파이프라인 의 스텝(Step) 생성
    - 4.1. 모델 빌딩 파이프라인 변수 생성
    - 4.2. 전처리 스텝 단계 정의
    - 4.3. 모델 학습을 위한 학습단계 정의
    - 4.4. 세이지 메이커 모델 생성 스탭 생성
    - 4.5. 실시간 엔드 포인트 배포 스텝 생성
- 5. 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의 및 실행
- 6. 세이지 메이커 스튜디오에서 실행 확인 하기
- 7. 아티펙트 경로 추출
    
---
### 노트북 커널
- 이 워크샵은 노트북 커널이 `conda_python3` 를 사용합니다. 다른 커널일 경우 변경 해주세요.
---



# 1. SageMaker 모델 빌드 파이프라인을 이용한 모델 빌드 오케스트레이션

Amazon SageMaker 모델 구축 파이프라인은 직접 SageMaker 통합을 활용하는 머신 러닝 파이프라인을 구축하기 위한 도구입니다. 

- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)

# 2. 파이프라인 개발자 가이드
- 상세 사항은 개발자 가이드 참조 하세요. --> [Amazon SageMaker 모델 구축 파이프라인](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/pipelines.html)


SageMaker 파이프라인은 다음 기능을 지원하며 본 노트북에서 하나씩 다루게 됩니다. 

* Processing job steps - 데이터처러 워크로드를 실행하기 위한 SageMaker의 관리형 기능. Feature engineering, 데이터 검증, 모델 평가, 모델 해석 등에 주로 사용됨 
* Training job steps - 학습작업. 모델에게 학습데이터셋을 이용하여 모델에게 예측을 하도록 학습시키는 반복적인 작업 
* Create model steps - 추론 엔드포인트 또는 배치 추론을 위한 모델의 생성 
* Pipelines - Workflow DAG. SageMaker 작업과 리소스 생성을 조율하는 단계와 조건을 가진다. 




In [ ]:
%load_ext autoreload
%autoreload 2

# 3. 기본 라이브러리 로딩 

- 세이지 메이커 관련 라이브러리를 로딩 합니다.

In [ ]:
import os
import boto3
import sagemaker
from time import strftime
from utils.ssm import parameter_store

In [ ]:
region = boto3.Session().region_name
pm = parameter_store(region)

prefix = pm.get_params(key="PREFIX")
bucket_name = pm.get_params(key="-".join([prefix, "BUCKET-NAME"]))

model_name = pm.get_params(key="-".join([prefix, "MODEL-NAME"]))
model_package_group_name = pm.get_params(key="-".join([prefix, "MODEL_PACKAGE_GROUP_NAME"]))

role = pm.get_params(key="-".join([prefix, "SAGEMAKER-ROLE-ARN"]))
role_arn = sagemaker.get_execution_role()
print(f"PREFIX : {prefix}")
print(f"BUCKET_NAME : {bucket_name}")
print(f"MODEL-NAME : {model_name}")
print(f"MODEL_PACKAGE_GROUP_NAME : {model_package_group_name}")
print(f"ROLE : {role}")

In [ ]:
local_mode = False

In [ ]:
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.pipeline_context import LocalPipelineSession
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="P1d")

if local_mode:
    # Create a `LocalPipelineSession` object so that each pipeline step will run locally
    # To run this pipeline in the cloud, you must change `LocalPipelineSession()` to `PipelineSession()`
    pipeline_session = LocalPipelineSession()
    git_config = None
else:
    pipeline_session = PipelineSession()
    
    git_config = {
        'repo': f'https://{pm.get_params(key="-".join([prefix, "CODE_REPO"]))}',
        'branch': 'main',
        'username': pm.get_params(key="-".join([prefix, "CODECOMMIT-USERNAME"]), enc=True),
        'password': pm.get_params(key="-".join([prefix, "CODECOMMIT-PWD"]), enc=True)
    }

# 4. 실시간 엔드 포인트 배포 스텝 생성

앤드포인트를 생성하기 위해서는 프로세싱 스텝을 통해서 합니다. 프레세싱 스텝에 앤드포인트 생성에 필요한 코드(스크립트)를 작성하여 프로세싱 스텝에서 실행하여 생성하게 합니다. 크게 아래와 같은 과정으로 합니다.

- 앤드포인트 생성 코드를 S3 에 업로드
- SKLearnProcessor 오브젝트 생성
- ProcessingStep 정의 (중요한 인자는 아래와 같습니다.)
    - processor (SKLearnProcessor 오브젝트 제공)
    - 코드에 전달할 커맨드 인자
        - endpoint config 생성시에, 이전 단계의 모델 결과를 제공합니다.
        - "--model_name", step_create_model.properties.ModelName,     
    - 앤드포인트 생성 코드



### 엔드포인트 생성 스크립트 코드를 S3에 업로딩

In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

In [ ]:
endpoint_instance_type = ParameterString(
    name="EndpointInstanceType",
    default_value="ml.m5.xlarge"
)

In [ ]:
from datetime import datetime
suffix = datetime.now().microsecond

### 프로세서 정의 및 스텝 정의

In [ ]:
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.processing import FrameworkProcessor

model_name = 'distilbert-base-uncased'

create_date = strftime("%m%d-%H%M%s")
job_name=f'endpoint-{model_name}-{create_date}'


deploy_model_processor = FrameworkProcessor(
    estimator_cls=PyTorch,
    framework_version="2.0.0",
    py_version='py310',
    instance_type="ml.t3.medium",
    instance_count=1,
    role=role,
    base_job_name=job_name,
    sagemaker_session=pipeline_session,
)

In [ ]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

proc_prefix = "/opt/ml/processing"
all_pipeline_endpoint_name = 'model-inference-' + str(suffix)

step_args = deploy_model_processor.run(
    code='deploy_model.py', #소스 디렉토리 안에서 파일 path
    source_dir= "./code", #현재 파일에서 소스 디렉토리 상대경로 # add processing.py and requirements.txt here
    git_config=git_config,
    arguments=[
        "--model_package_group_name", model_package_group_name, 
        "--region", region,
        "--endpoint_instance_type", endpoint_instance_type,
        "--role", role_arn
    ]
)
step_deploy = ProcessingStep(
    name="DeployBertData",
    step_args=step_args,
    # cache_config=cache_config
)

## 4.1 배포 파이프라인 정의
- 위에서 정의한 파라미터를 제공
- 실행할 스텝 기술
    - steps=[step_process, step_train, step_create_model, step_deploy],
- 아래는 약 20분 정도 소요 됩니다.

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

deploy_pipeline = Pipeline(
    name="BertModelDeployPipeline",
    parameters=[
        endpoint_instance_type
    ],
   steps=[step_deploy],

)

## 4.2 배포 파이프라인 정의 확인
위에서 정의한 파이프라인 정의는 Json 형식으로 정의 되어 있습니다.

In [ ]:
import json

deploy_definition = json.loads(deploy_pipeline.definition())
deploy_definition

## 4.3 배포 파이프라인 정의를 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [ ]:
deploy_pipeline.upsert(role_arn=role_arn)
deploy_execution = deploy_pipeline.start()

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [ ]:
deploy_execution.describe()

## 4.4 배포 파이프라인 실행 기다리기

In [ ]:
deploy_execution.wait()

실행이 완료될 때까지 기다립니다.

## 4.5 배포 파이프라인 실행 단계 기록 보기

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [ ]:
deploy_execution.list_steps()

## 5. Create Amazon EventBridge Rule
model registry에서 모델이 Approved되었을 때 이벤트 트리거를 만들기 위한 설정을 Amazon EventBridge Rule을 이용하여 설정합니다.

- arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess 의 권한 설정을 추가해야 합니다.


In [ ]:
iam_client = boto3.client('iam')

base_role_name=role

iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonEventBridgeFullAccess'
)

In [ ]:
event_client = boto3.client('events')

In [ ]:
eventpattern = json.dumps(
    {
      "source": ["aws.sagemaker"],
      "detail-type": ["SageMaker Model Package State Change"],
      "detail": {
        "ModelPackageGroupName": [f"{model_package_group_name}"],
        "ModelApprovalStatus": ["Approved"]
      }
    }
)

In [ ]:
rule_name = 'bert_model_package_state'
event_rule = event_client.put_rule(
    Name=rule_name,
    EventPattern=eventpattern,
    State='ENABLED',
    Description='This is after the approval update for the bert model',
)

In [ ]:
iam_client = boto3.client('iam')

base_role_name = 'Amazon_EventBridge_Invoke_SageMaker_Pipeline'
response = iam_client.create_role(
    RoleName=base_role_name,
    AssumeRolePolicyDocument=json.dumps(
        {'Version': '2012-10-17',
         'Statement': [
             {'Effect': 'Allow',
               'Principal': {'Service': 'events.amazonaws.com'},
              'Action': 'sts:AssumeRole'}
         ]
        }
    )
)

PolicyDocument = json.dumps({
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker:StartPipelineExecution"
            ],
            "Resource": [
                deploy_pipeline.describe()['PipelineArn']
            ]
        }
    ]
})

response = iam_client.create_policy(
    PolicyName='Amazon_EventBridge_Invoke_SageMaker_Pipeline',
    Path='/service-role/',
    PolicyDocument=PolicyDocument
)

base_role_name=role

iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn=response['Policy']['Arn']
)

In [ ]:
event_client.put_targets(
    Rule=rule_name,
    Targets=[
        {
            'Id': 'Target0',
            'Arn': deploy_pipeline.describe()['PipelineArn'],
            'RoleArn': role_arn
        }
    ]
)